# NOAA Retrieve Files via FTP
#### Description:
This notebook is used to retrieve NOAA Daily Files through a public FTP connection and place them in the current working directory.

Source: https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/

#### Files Retrieved:
- ghcnd-stations.txt
- ghcnd-states.txt
- ghcnd-countries.txt
- ghcnd-inventory.txt

#### Created by:
Nate Muth <br>
nmuth87@gmail.com

#### Created on:
7/29/2018

#### Changelog:
7/29/2018 - Initial Create Date<br>
8/5/2018 - Added lists of files to download in each directory in the config section<br>
8/9/2018 - Updated to use NOAA FTP wrapper; deleted old

In [1]:
# CONFIG

# metadataFiles are stored in the root folder
metadataFiles = ['ghcnd-stations.txt' # Retrieve stations file
                 ,'ghcnd-states.txt' # Retrieve states file
                 ,'ghcnd-countries.txt' # Retrieve countries file
                 ,'ghcnd-inventory.txt' # Retrieve inventory file
                 ]

In [2]:
# pull files for main folder
import pyFTP_NOAA
ftp = pyFTP_NOAA.ftp_noaa()

for f in metadataFiles:
    ftp.retrieveFile(f)
    
ftp.quit()